In [1]:
import numpy as np

from triqs.gf import *

from itertools import product

from hugoDimer import GWHubbardDimer
from gwsolver import GWSolver

Starting serial run at: 2023-11-19 18:15:02.481055


In [2]:
def ifmesh(beta, statistic, nw):
    if statistic == 'Fermion':
        wmax = (2 * (nw - 1) + 1) * np.pi / beta
        return np.linspace(-wmax, wmax, 2 * nw) * 1j
    elif statistic == 'Boson':
        wmax = (2 * (nw - 1)) * np.pi / beta
        mesh_p = np.linspace(0, wmax, nw) * 1j
        mesh_m = (-np.linspace(0, wmax, nw) * 1j)[::-1]
        return np.concatenate((mesh_m[:-1], mesh_p))

def get_data(G):
    if isinstance(G, BlockGf):
         return G['up']
    return G

def G0(spinless, mesh, t):
    g0_w = Gf(mesh=mesh, target_shape = [2, 2])
    for w in mesh:
        g0_w[w] = 0.5 *np.array([
            [1 / (w - t) + 1 / (w + t), -1 / (w - t) + 1 / (w + t)],
            [-1 / (w - t) + 1 / (w + t), 1 / (w - t) + 1 / (w + t)]
        ])
    if spinless:
        return g0_w
    else:
        return BlockGf(block_list = [g0_w] * 2, name_list = ('up', 'dn'))

def polarizationCheck(myPolarization, hugoPolarization):
    np.testing.assert_array_almost_equal(hugoPolarization[:, Idx(0, 0, 0)][0,0,0,0].data, myPolarization.data[:, 0, 0, 0, 0])
    np.testing.assert_array_almost_equal(hugoPolarization[:, Idx(1, 0, 0)][0,0,0,0].data, myPolarization.data[:, 1, 1, 0, 0])
    
def screenedPotentialCheck(myPotential, hugoPotential):
    np.testing.assert_array_almost_equal(hugoPotential[:, Idx(0, 0, 0)][0,0,0,0].data, myPotential.data[:, 0, 0, 0, 0])
    np.testing.assert_array_almost_equal(hugoPotential[:, Idx(1, 0, 0)][0,0,0,0].data, myPotential.data[:, 1, 1, 0, 0])

def selfEnergyCheck(myEnergy, hugoEnergy):
    np.testing.assert_array_almost_equal(hugoEnergy[:, Idx(0, 0, 0)][0,0].data, myEnergy.data[:, 0, 0])
    np.testing.assert_array_almost_equal(hugoEnergy[:, Idx(1, 0, 0)][0,0].data, myEnergy.data[:, 0, 1])

def GWCheck(myGW, hugoGW):
    np.testing.assert_array_almost_equal(hugoGW[:, Idx(0, 0, 0)][0,0].data, myGW.data[:, 0, 0])
    np.testing.assert_array_almost_equal(hugoGW[:, Idx(1, 0, 0)][0,0].data, myGW.data[:, 0, 1])
    
def compare(hartree_flag, fock_flag, spinless):
    beta = 100.0
    nw = 1024
    t = 1.0
    U = 1.5
    fmesh = MeshImFreq(beta, 'Fermion', nw)

    V = np.zeros((2, 2, 2, 2))
    V[0, 0, 0, 0] = U
    V[1, 1, 1, 1] = U
        
    g0_w = G0(spinless, fmesh, t)
        
    gw = GWSolver(g0_w = g0_w, V = V, hartree_flag = hartree_flag, fock_flag = fock_flag)

    gwa = GWHubbardDimer(
        beta = beta, U = U,
        t = t, nw = nw, maxiter = 1,
        self_interaction = True, spinless = spinless,
        hartree_flag = hartree_flag, fock_flag = fock_flag)
    
    P = get_data(gw.P_w)
    W = get_data(gw.W_w)
    sigma = get_data(gw.sigma_w)
    g_w = get_data(gw.g_w)

    Pe = gwa.P_wr
    We = gwa.W_wr
    sigma_e = gwa.sigma_wr
    gwe = gwa.g_wr
    
    print("-----------------------------")
    print("GW Implementation Test:")
    lines = [
        "\t Hartree flag : " + str(hartree_flag),
        "\t Fock flag : " + str(fock_flag),
        "\t spinless : " + str(spin_flag)
    ]
    max_length = max(line.index(':') for line in lines)
    for line in lines:
        left_part, right_part = line.split(':')
        formatted_line = f"{left_part.rstrip().ljust(max_length)} : {right_part.strip()}"
        print(formatted_line)
    print("\n")

    
    polarizationCheck(P, Pe)
    screenedPotentialCheck(W, We)
    selfEnergyCheck(sigma, sigma_e)
    GWCheck(g_w, gwe)

    lines = [
        "Polarization : Passed",
        "Screened Potential : Passed",
        "Self Energy : Passed",
        "GW Green function : Passed"
    ]
    max_length = max(line.index(':') for line in lines)
    for line in lines:
        left_part, right_part = line.split(':')
        formatted_line = f"{left_part.rstrip().ljust(max_length)} : {right_part.strip()}"
        print(formatted_line)
    print("-----------------------------")

flags = [True, False]
for hartree_flag, fock_flag, spin_flag in product(flags, flags, flags):
    compare(hartree_flag = hartree_flag, fock_flag = fock_flag, spinless = spin_flag)

-----------------------------
GW Implementation Test:
	 Hartree flag  : True
	 Fock flag     : True
	 spinless      : True


Polarization        : Passed
Screened Potential  : Passed
Self Energy         : Passed
GW Green function   : Passed
-----------------------------
-----------------------------
GW Implementation Test:
	 Hartree flag  : True
	 Fock flag     : True
	 spinless      : False


Polarization        : Passed
Screened Potential  : Passed
Self Energy         : Passed
GW Green function   : Passed
-----------------------------
-----------------------------
GW Implementation Test:
	 Hartree flag  : True
	 Fock flag     : False
	 spinless      : True


Polarization        : Passed
Screened Potential  : Passed
Self Energy         : Passed
GW Green function   : Passed
-----------------------------
-----------------------------
GW Implementation Test:
	 Hartree flag  : True
	 Fock flag     : False
	 spinless      : False


Polarization        : Passed
Screened Potential  : Passed
